In [83]:
import pandas as pd
import tensorflow as tf
train = pd.read_csv("train_data.csv")
train.drop(["Id","date"],axis=1,inplace=True)
target = pd.read_csv("train_target.csv")
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
values = train.values
s.fit(values)
values = s.transform(values)
labels = target["target"].to_numpy()

In [84]:
labels,values.shape

(array([1, 0, 1, ..., 1, 0, 0], dtype=int64), (25500, 16))

In [85]:
values.shape

(25500, 16)

In [51]:
values = values.transpose()

In [86]:
values.shape

(25500, 16)

In [87]:
values = tf.expand_dims(values,axis=2).numpy()
values.shape

(25500, 16, 1)

In [5]:
df = pd.read_csv("TRAINING_SET_FINAL.csv")
df.drop(["date","Id"],axis=1,inplace=True)
val = df.values
s.fit(val)
val = s.transform(val)
val.shape,val

((25467, 28),
 array([[-0.76879334, -0.77316675, -0.76903088, ..., -0.27051434,
          1.13039316, -0.84441121],
        [-0.77477854, -0.77471512, -0.7730916 , ..., -0.36764203,
          1.10267818, -0.842573  ],
        [-0.77176501, -0.76927493, -0.76656095, ..., -0.37658546,
          1.07429041, -0.84042617],
        ...,
        [ 1.31530979,  1.31276994,  1.31993871, ...,  0.77634292,
         -0.43043442,  1.30743924],
        [ 1.31330077,  1.31214222,  1.31993871, ...,  0.70935008,
         -0.37712967,  1.30771367],
        [ 1.31564462,  1.31260255,  1.32106902, ...,  0.64805223,
         -0.32843623,  1.30860567]]))

In [109]:
from tensorflow.keras.layers import Input, Conv1D, Dense, Lambda, Flatten, Concatenate
from tensorflow.keras import Model
from tensorflow.keras import metrics
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras import utils
from sklearn.preprocessing import StandardScaler
import numpy as np
import math

        
inputs = Input(shape=(16,1))

# extract our features using a Convolutional layers, hence "CNN"
feature_extraction = Conv1D(64, 5, activation='relu')(inputs)
# split layer into three regions based on time, hence "Temporal"
long_term = Lambda( lambda x: tf.split(x, num_or_size_splits=3, axis=1)[0])(feature_extraction)
mid_term = Lambda( lambda x: tf.split(x, num_or_size_splits=3, axis=1)[1])(feature_extraction)
short_term = Lambda( lambda x: tf.split(x, num_or_size_splits=3, axis=1)[2])(feature_extraction)

long_term_conv = Conv1D(1, 1, activation='relu')(long_term)
mid_term_conv = Conv1D(1, 1, activation='relu')(mid_term)
short_term_conv = Conv1D(1, 1, activation='relu')(short_term)

# combine three layers back into one
combined = Concatenate(axis=1)([long_term_conv, mid_term_conv, short_term_conv])

# flattening is required since our input is a 2D matrix
flattened = Flatten()(combined)

# 1 output neuron for each class (Up, Stationary, Down --- see Direction class)
x = Dense(20, activation='softmax')(flattened)
outputs = Dense(1,activation="sigmoid")(x)
# specify input and output layers of our model
model = Model(inputs=inputs, outputs=outputs)

# compile our model
model.compile(optimizer='adam',
              loss="binary_crossentropy",metrics="accuracy")

#     def __PrepareData(self, data, rolling_avg_window_size=5, stationary_threshold=.0001):
#         '''Prepares the data for a format friendly for our model'''

#         # rolling_avg_window_size = window size for the future mid prices to average, 
#         #   this average is what the model wants to predict
#         # stationary_threshold = maximum change of movement to be considered stationary 
#         #   for the average mid price stated above 

#         df = data[input_vars]
#         shift = -(rolling_avg_window_size-1)

#         # function we will use to label our data (used in line )
#         def label_data(row):
#             if row['close_avg_change_pct'] > stationary_threshold:
#                 return Direction.UP
#             elif row['close_avg_change_pct'] < -stationary_threshold:
#                 return Direction.DOWN
#             else:
#                 return Direction.STATIONARY

#         # compute the % change in the average of the close of the future 5 time steps
#         #   at each time step
#         df['close_avg'] = df['close'].rolling(window=rolling_avg_window_size).mean().shift(shift) 
#         df['close_avg_change_pct'] = (df['close_avg'] - df['close']) / df['close']

#         # label data based on direction,
#         # axis=1 signifies a row-wise operation (axis=0 is col-wise)
#         df['movement_labels'] = df.apply(label_data, axis=1)

#         # lists to store each 2D input matrix and the corresponding label
#         data = []
#         labels = []

#         for i in range(len(df)-self.n_tsteps+1+shift):
#             label = df['movement_labels'].iloc[i+self.n_tsteps-1]
#             data.append(df[input_vars].iloc[i:i+self.n_tsteps].values)
#             labels.append(label)

#         data = np.array(data)

#         # temporarily reshape data to 2D,
#         #   necessary because sklearn only works wtih 2D data
#         dim1, dim2, dim3 = data.shape
#         data = data.reshape(dim1*dim2, dim3)

#         # fit our scaler and transform our data in one method call
#         data = self.scaler.fit_transform(data)

#         # return data to original shape
#         data = data.reshape(dim1, dim2, dim3)

#         # Keras needs dummy matrices for classification problems, 
#         #   hence the need for to_categorical()
#         #   num classes ensures our dummy matrix has 3 columns, 
#         #   one for each label (Up, Down, Stationary)
#         return data, utils.to_categorical(labels, num_classes=3)

model.fit(values, labels, epochs=50)


Epoch 1/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6928 - accuracy: 0.5045
Epoch 2/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6925 - accuracy: 0.5151
Epoch 3/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6924 - accuracy: 0.5170
Epoch 4/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6923 - accuracy: 0.5172
Epoch 5/50
797/797 [==============================] - 7s 8ms/step - loss: 0.6923 - accuracy: 0.5165
Epoch 6/50
797/797 [==============================] - 8s 10ms/step - loss: 0.6923 - accuracy: 0.5180
Epoch 7/50
797/797 [==============================] - 9s 11ms/step - loss: 0.6922 - accuracy: 0.5186
Epoch 8/50
797/797 [==============================] - 9s 11ms/step - loss: 0.6922 - accuracy: 0.5177
Epoch 9/50
797/797 [==============================] - 9s 11ms/step - loss: 0.6921 - accuracy: 0.5185
Epoch 10/50
797/797 [==============================] - 9s 11ms/step - loss: 0.6921 - accuracy: 0

In [77]:
model.fit(values, labels, epochs=50)

Epoch 1/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6913 - accuracy: 0.5206
Epoch 2/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6912 - accuracy: 0.5218
Epoch 3/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6913 - accuracy: 0.5221
Epoch 4/50
797/797 [==============================] - 4s 6ms/step - loss: 0.6911 - accuracy: 0.5250
Epoch 5/50
797/797 [==============================] - 4s 6ms/step - loss: 0.6912 - accuracy: 0.5216
Epoch 6/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6912 - accuracy: 0.5232
Epoch 7/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6912 - accuracy: 0.5222
Epoch 8/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6911 - accuracy: 0.5247
Epoch 9/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6911 - accuracy: 0.5271
Epoch 10/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6910 - accuracy: 0.5249

In [102]:
test = pd.read_csv("test_data.csv")
test.drop(["Id","date"],axis=1,inplace=True)
test = test.values
test

array([[0.6034    , 0.60345625, 0.6010875 , ..., 0.60194594, 0.60562733,
        0.59826455],
       [0.6011    , 0.60208125, 0.600225  , ..., 0.60210344, 0.60522458,
        0.59898229],
       [0.60053125, 0.6020375 , 0.60048125, ..., 0.6020525 , 0.60520933,
        0.59889567],
       ...,
       [0.7061    , 0.71451875, 0.7061    , ..., 0.70218531, 0.70916219,
        0.69520843],
       [0.7134125 , 0.7157    , 0.7118875 , ..., 0.70294031, 0.71161722,
        0.69426341],
       [0.71443125, 0.7144625 , 0.7119875 , ..., 0.70380531, 0.71307354,
        0.69453708]])

In [103]:
test = tf.expand_dims(test,axis=2)
test.shape

TensorShape([10317, 16, 1])

In [104]:
test = test.numpy()
test.shape

(10317, 16, 1)

In [110]:
pred = tf.round(model.predict(test)).numpy()
pred

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [111]:
pred = tf.cast(pred,dtype=tf.int64)
pred

<tf.Tensor: shape=(10317, 1), dtype=int64, numpy=
array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int64)>

In [74]:
l = np.arange(0,len(test))
l,l.shape

(array([    0,     1,     2, ..., 10314, 10315, 10316]), (10317,))

In [112]:
pred = tf.squeeze(pred).numpy()
pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [113]:
pred_df = pd.DataFrame(list(zip(l, pred)),
               columns =['Id', 'target'])
pred_df.to_csv("submission9.csv")
pred_df["target"].value_counts()

1    10316
0        1
Name: target, dtype: int64